<a href="https://colab.research.google.com/github/AnuruddhaPaul/CONVOLUTION-_From_Scratch/blob/main/CONVOLUTION__From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install Layer

In [ ]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        # TODO: return output
        pass

    def backward(self, output_gradient, learning_rate):
        # TODO: update parameters and return input gradient
        pass

In [ ]:
import numpy as np
from scipy import signal
from layer import Layer

class Convolutional(Layer):
    def __init__(self, input_shape, kernel_size, depth):
        """
        input_shape: tuple (input_depth, input_height, input_width)
        kernel_size: int (size of the square filter, e.g., 3 for 3x3)
        depth: int (number of filters/kernels to learn)
        """
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth

        # Calculate output dimensions based on 'valid' padding (no zero padding)
        # Formula: Output = Input - Kernel + 1
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)

        # 4D Shape for kernels: (Number of Filters, Input Depth, Height, Width)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)

        # Initialize weights and biases with random values
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input):
        self.input = input # Cache input for backward pass
        self.output = np.copy(self.biases) # Start with biases

        # Loop over every output filter (i)
        for i in range(self.depth):
            # Loop over every input channel (j)
            for j in range(self.input_depth):
                # Correlate input channel j with kernel channel j
                # 'valid' means the kernel stays strictly inside the image boundaries
                # Accumulate (+=) results across all input channels for this filter
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid")
        return self.output

    def backward(self, output_gradient, learning_rate):
        # Initialize empty gradients
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                # Calculate Gradient for Kernels (Weights)
                # Cross-correlate input with output error to see which weight caused the error
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")

                # Calculate Gradient for Inputs (to pass to previous layer)
                # 'full' convolution expands the gradient back to the original input size
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")

        # Update weights using Gradient Descent
        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient

        return input_gradient

In [ ]:
import numpy as np
from layer import Layer

class Activation(Layer):
    def __init__(self, activation, activation_prime):
        """
        activation: The function to apply (e.g., numpy.tanh)
        activation_prime: The derivative of that function (e.g., 1 - tanh^2)
        """
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        # Cache the input because the derivative calculation depends on it
        self.input = input
        # Apply the function to every element in the input matrix
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        """
        Computes the gradient to pass back to the previous layer.
        Formula: Local Gradient * Output Gradient (Chain Rule)
        """
        # np.multiply performs element-wise multiplication
        # We multiply the incoming error (output_gradient) by the slope of the function (activation_prime)
        return np.multiply(output_gradient, self.activation_prime(self.input))

In [ ]:
import numpy as np
from layer import Layer
from activation import Activation

class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_prime(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_prime)

class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)

class Softmax(Layer):
    def forward(self, input):
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        return self.output

    def backward(self, output_gradient, learning_rate):
        # This version is faster than the one presented in the video
        n = np.size(self.output)
        return np.dot((np.identity(n) - self.output.T) * self.output, output_gradient)
        # Original formula:
        # tmp = np.tile(self.output, n)
        # return np.dot(tmp * (np.identity(n) - np.transpose(tmp)), output_gradient)

In [ ]:
import numpy as np
from layer import Layer

class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient

In [ ]:
import numpy as np

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)

def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils

from dense import Dense
from convolutional import Convolutional
from reshape import Reshape
from activations import Sigmoid
from losses import binary_cross_entropy, binary_cross_entropy_prime
from network import train, predict

def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)

# neural network
network = [
    Convolutional((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]

# train
train(
    network,
    binary_cross_entropy,
    binary_cross_entropy_prime,
    x_train,
    y_train,
    epochs=20,
    learning_rate=0.1
)

# test
for x, y in zip(x_test, y_test):
    output = predict(network, x)
    print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")